# This is Nicole's Attempt to Scrape Craig


Let's do thissss

In [1]:
#import get to call a get request on the site
from requests import get

#get the first page of the east bay housing prices
response = get('https://losangeles.craigslist.org/search/sfv/apa') #get rid of those lame-o's that post a housing option without a pic using their filter

from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')
print(type(posts)) #to double check that I got a ResultSet
print(len(posts)) #to double check I got 120 (elements/page)

<class 'bs4.element.ResultSet'>
120


In [ ]:
#build out the loop
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []

for page in pages:
    
    #get request
    response = get("https://losangeles.craigslist.org/search/lgb/apa?" 
                   + "s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
#                    + "&hasPic=1"
#                    + "&availabilityMode=0"
                  )

    sleep(randint(1,5))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #define the posts
    posts = page_html.find_all('li', class_= 'result-row')
        
    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            #posting date
            #grab the datetime element 0 for date and 1 for time
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            post_price = post.a.text.strip().replace("$", "")
            post_prices.append(post_price)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #    bedroom_counts.append(bedroom_count)
                
            #    sqft = np.nan
            #    sqfts.append(sqft)
                
    iterations += 1
    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

In [3]:
import pandas as pd

SFV_apts = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices})

In [4]:
SFV_apts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3120 entries, 0 to 3119
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   posted           3120 non-null   object 
 1   neighborhood     3120 non-null   object 
 2   post title       3120 non-null   object 
 3   number bedrooms  2522 non-null   object 
 4   sqft             2574 non-null   float64
 5   URL              3120 non-null   object 
 6   price            3120 non-null   object 
dtypes: float64(1), object(6)
memory usage: 170.8+ KB


In [5]:
SFV_apts.head()

,posted,neighborhood,post title,number bedrooms,sqft,URL,price
0,2022-05-22 09:49,(Van Nuys),❖ Nice Studio in Van Nuys *Full Kitchen *2 Clo...,NaN,425.0,https://losangeles.craigslist.org/sfv/apa/d/va...,"1,575"
1,2022-05-22 09:49,(North Hollywood),Renovated Studio in NoHo +Full Kitchen +Plank ...,NaN,500.0,https://losangeles.craigslist.org/sfv/apa/d/to...,"1,795"
2,2022-05-22 09:49,(Northridge),2+2 in Northridge +SS Appliances +Quartz Count...,2,945.0,https://losangeles.craigslist.org/sfv/apa/d/no...,"2,750"
3,2022-05-22 09:48,(Glendale),Glendale Newly Renovated 2+1 with SS Appliance...,2,875.0,https://losangeles.craigslist.org/sfv/apa/d/gl...,"2,450"
4,2022-05-22 09:47,(Toluca Lake),LIVE IN BEAUTIFUL TOLUCA LAKE-SPACIOUS 1+1 & L...,1,847.0,https://losangeles.craigslist.org/sfv/apa/d/no...,"3,455"


In [7]:
SFV_apts.to_csv('data/SFV_apts_May22.csv')

In [16]:
SFV_apts.URL[0]

'https://losangeles.craigslist.org/sfv/apa/d/canoga-park-room-for-rent-brand-new/7485745558.html'

In [10]:
''''# this code is from Seth!
import requests

def get_latlong(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    try:
        link = soup.find_all('div', class_='viewposting')
        lat = link[0]['data-latitude']
        long = link[0]['data-longitude']
        time.sleep(random.uniform(.5,3))
    #this code could be used to randomize the time between calls to craigslist and slow down the scraper 
    #It definitely helps to keep from getting blocked/throttled
        return [lat, long]
    except:
        return [None, None]

In [17]:
SFV_apts['latlon'] = SFV_apts.URL.apply(get_latlong)

In [18]:
SFV_apts.head(30)

,posted,neighborhood,post title,number bedrooms,sqft,URL,price,latlon
0,2022-05-21 20:48,(Canoga park),Room for rent brand new renovation,1,NaN,https://losangeles.craigslist.org/sfv/apa/d/ca...,,"[None, None]"
1,2022-05-21 20:29,(Woodland Hills),Beautiful Loft Style 1x1 for May Move-ins,1,717.0,https://losangeles.craigslist.org/sfv/apa/d/wo...,"2,510","[None, None]"
2,2022-05-21 20:28,"(18414 W. Jakes Way, Canyon Country, CA)","Lounging and Sunbathing Areas, Picnic Areas, A...",2,919.0,https://losangeles.craigslist.org/sfv/apa/d/ca...,"2,400","[None, None]"
3,2022-05-21 20:25,(OPEN HOUSE DAILY 10-5 | Northridge),NORTHRIDGE 2 Bedroom - Close to CSUN - Clubhou...,2,1044.0,https://losangeles.craigslist.org/sfv/apa/d/no...,"2,505","[None, None]"
4,2022-05-21 20:24,(Sherman Oaks / South of Ventura Blvd),South of the Blvd ^ Pool ^ 1 Bedroom With Loft...,1,850.0,https://losangeles.craigslist.org/sfv/apa/d/sh...,"2,595","[None, None]"
5,2022-05-21 20:24,(Near Ventura Blvd),1 Bedroom 1 BA in Encino | BBQ | Gym | Heated ...,1,965.0,https://losangeles.craigslist.org/sfv/apa/d/en...,"2,395","[None, None]"
6,2022-05-21 20:24,(Sherman Oaks 91335),Pool | Pet-Friendly | Picnic Area With BBQ | S...,NaN,429.0,https://losangeles.craigslist.org/sfv/apa/d/re...,"1,790","[None, None]"
7,2022-05-21 20:24,(North Hills 91343),2 Bedroom in North Hills with Balcony++ Lamina...,2,1100.0,https://losangeles.craigslist.org/sfv/apa/d/no...,"2,200","[None, None]"
8,2022-05-21 20:23,(Northridge / Close to CSUN),Plank Flooring | Pool | Quartz | 1 Bedroom in ...,1,850.0,https://losangeles.craigslist.org/sfv/apa/d/no...,"1,850","[None, None]"
9,2022-05-21 20:23,(Van Nuys 91405),Plank Floors | Pool | Air Conditioning | 1 Bed...,1,700.0,https://losangeles.craigslist.org/sfv/apa/d/pl...,"1,950","[None, None]"
